In [104]:
import numpy as np
import os
from pathlib import Path
from typing import Union, List
from tqdm import tqdm

In [111]:
def read_signals(infile: Union[str, List[str]]) -> np.ndarray:
    """
    Read signals from a .npy file, a list of .npy file paths, or a directory containing .npy files.
    
    Args:
        infile (str | list[str]): Path to a .npy file, a list of .npy file paths, or a directory.
        
    Returns:
        np.ndarray: The concatenated signals.
    """
    file_paths = []
    if isinstance(infile, str):
        path = Path(infile)
        if path.is_file() and path.suffix == '.npy':
            file_paths = [path]
        elif path.is_dir():
            file_paths = sorted([f for f in path.glob("*.npy") if f.is_file()])
        else:
            raise ValueError(f"Input '{infile}' is neither a valid .npy file nor a directory.")
    elif isinstance(infile, list):
        file_paths = [Path(f) for f in infile if Path(f).is_file() and Path(f).suffix == '.npy']
        file_paths = sorted(file_paths)
    else:
        raise ValueError("infile must be a string or a list of strings.")

    if not file_paths:
        raise ValueError("No valid .npy files found to process.")

    signals = []
    for file in tqdm(file_paths, desc="Loading files", unit="file"):
        try:
            data = np.load(file, allow_pickle=True)
            if data.shape[0] != 0:
                if data.shape[0] == 1:
                    data = np.squeeze(data, axis=0)
                else:
                    raise ValueError(f"File {file} contains data with incompatible dimensions.")
            signals.append(data)
            print(f"Loaded file: {file}")
        except FileNotFoundError:
            print(f"Error: File {file} not found.")
        except ValueError as ve:
            print(f"Error: File {file} is not a valid numpy array: {ve}")
        except Exception as e:
            print(f"Error: Error loading file {file}: {e}")

    if not signals:
        raise ValueError("No valid .npy files were loaded.")

    if len(signals) == 1:
        return signals[0]

    max_cols = max(arr.shape[1] for arr in signals)
    total_rows = sum(arr.shape[0] for arr in signals)
    output = np.empty((total_rows, max_cols), dtype=np.float64)

    row_idx = 0
    for arr in signals:
        rows, cols = arr.shape
        arr_padded = np.pad(arr, ((0, 0), (0, max_cols - cols)), mode='constant', constant_values=np.nan) if cols < max_cols else arr
        output[row_idx:row_idx + rows, :] = arr_padded
        row_idx += rows

    return output
    
test1 = read_signals("sample_data/T2_PGa-T2_FOV1_LOW-D9_behavior_extractedsignals_raw.npy")
print(f"Test1 shape: {test1.shape}")

test2 = read_signals(["sample_data/T2_IG-19_2P-HER_PFC-FOV2_LOW-D10_behavior-000_extractedsignals_raw.npy", "sample_data/T2_LCDD-PGa4_FOV1_LOW_D10_behavior-004_extractedsignals_raw.npy", "sample_data/T2_PGa-T2_FOV1_LOW-D9_behavior_extractedsignals_raw.npy"])
print(f"Test2 shape: {test2.shape}")

test3 = read_signals("sample_data")
print(f"Test3 shape: {test3.shape}")

Loading files:   0%|          | 0/1 [00:00<?, ?file/s]/tmp/ipykernel_826154/3500667171.py:32: UserWarning: Reading `.npy` or `.npz` file required additional header parsing as it was created on Python 2. Save the file again to speed up loading and avoid this warning.
  data = np.load(file, allow_pickle=True)
Loading files: 100%|██████████| 1/1 [00:00<00:00, 223.49file/s]


Loaded file: sample_data/T2_PGa-T2_FOV1_LOW-D9_behavior_extractedsignals_raw.npy
Test1 shape: (120, 25371)


Loading files: 100%|██████████| 3/3 [00:00<00:00, 302.18file/s]


Loaded file: sample_data/T2_IG-19_2P-HER_PFC-FOV2_LOW-D10_behavior-000_extractedsignals_raw.npy
Loaded file: sample_data/T2_LCDD-PGa4_FOV1_LOW_D10_behavior-004_extractedsignals_raw.npy
Loaded file: sample_data/T2_PGa-T2_FOV1_LOW-D9_behavior_extractedsignals_raw.npy
Test2 shape: (280, 54000)


Loading files: 100%|██████████| 3/3 [00:00<00:00, 375.34file/s]

Loaded file: sample_data/T2_IG-19_2P-HER_PFC-FOV2_LOW-D10_behavior-000_extractedsignals_raw.npy
Loaded file: sample_data/T2_LCDD-PGa4_FOV1_LOW_D10_behavior-004_extractedsignals_raw.npy
Loaded file: sample_data/T2_PGa-T2_FOV1_LOW-D9_behavior_extractedsignals_raw.npy
Test3 shape: (280, 54000)
